In [1]:
import json
import numpy as np
import random
from sklearn.metrics import mean_absolute_error, mean_squared_error
from collections import defaultdict, OrderedDict
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from string import punctuation


import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def readJson(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return df

df = readJson('assignment1/train.json.gz')
# np.random.seed(1)
# np.random.shuffle(df)

trainingData = [df[x] for x in range(0,150000)]
validationData = [df[x] for x in range(150000,200000)]

In [2]:
userIndex = {}
users = []
userToItem = defaultdict(list)
itemIndex = {}
items = []
itemToUser = defaultdict(list)
i,j = 0,0
trainLines,validationLines = [],[]
ID = 0
for data in trainingData:
    userToItem[data['reviewerID']].append(data['itemID'])
    itemToUser[data['itemID']].append(data['reviewerID'])
    if data['reviewerID'] not in userIndex:
        userIndex[data['reviewerID']] = i
        users.append(data['reviewerID'])
        i += 1
    if data['itemID'] not in itemIndex:
        itemIndex[data['itemID']] = j
        items.append(data['itemID'])
        j += 1
    trainLines.append(str(ID)+','+str(userIndex[data['reviewerID']])+','+str(itemIndex[data['itemID']])+','+str(int(data['rating'])))
    ID += 1

for data in validationData:
    userToItem[data['reviewerID']].append(data['itemID'])
    itemToUser[data['itemID']].append(data['reviewerID'])
    if data['reviewerID'] not in userIndex:
        userIndex[data['reviewerID']] = i
        users.append(data['reviewerID'])
        i += 1
    if data['itemID'] not in itemIndex:
        itemIndex[data['itemID']] = j
        items.append(data['itemID'])
        j += 1
    validationLines.append(str(ID)+','+str(userIndex[data['reviewerID']])+','+str(itemIndex[data['itemID']]))
    ID += 1

# Rui = np.zeros((len(users), len(items)))
# for data in trainingData:
#     Rui[userIndex[data['reviewerID']]][itemIndex[data['itemID']]] = data['rating']

In [3]:
with open('train.txt', mode='wt') as myfile:
    myfile.write('ID,user,movie,rating\n')
    myfile.write('\n'.join(trainLines))
with open('validation.txt', mode='wt') as myfile:
    myfile.write('ID,user,movie,rating\n')
    myfile.write('\n'.join(validationLines ))

In [4]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD

pd.options.display.max_columns = 10 
pd.options.display.width = 134
pd.options.display.max_rows = 20

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

train = pd.read_csv('train.txt')
test = pd.read_csv('validation.txt')
matrix = pd.concat([train,test]).pivot('user','movie','rating')
movie_means = matrix.mean()
user_means = matrix.mean(axis=1)
mzm = matrix-movie_means
# mz = mzm.fillna(0)
# mask = -mzm.isnull()

In [5]:
mz = mzm.fillna(0)
mask = -mzm.isnull()

MemoryError: 

In [ ]:
iteration = 0
mse_last = 999
while iteration<10:
    iteration += 1
    svd = TruncatedSVD(n_components=15,random_state=42)
    svd.fit(mz)
    print 'here',
    mzsvd = pd.DataFrame(svd.inverse_transform(svd.transform(mz)),columns=mz.columns,index=mz.index)
    print 'here2',
#     mse = mean_squared_error(mzsvd[mask].fillna(0),mzm[mask].fillna(0))
#     print('%i %.5f %.5f'%(iteration,mse,mse_last-mse))
    mzsvd[mask] = mzm[mask]

    mz = mzsvd
#     if mse_last-mse<0.00001: break
#     mse_last = mse

# m = mz+movie_means
# m = m.clip(lower=1,upper=5)

# test['rating'] = test.apply(lambda x:m[m.index==x.user][x.movie].values[0],axis=1)

# # There are some movies who did not have enough info to make prediction, so just used average value for user
# missing = np.where(test.rating.isnull())[0]
# test.ix[missing,'rating'] = user_means[test.loc[missing].user].values

# test.to_csv('submission.csv',index=False,columns=['ID','rating'])

here here2

In [ ]:
for line in open("assignment1/pairs_Rating.txt"):
    if line.startswith("userID"):
        continue
    user,item = line.strip().split('-')
    if user not in userIndex:
        userIndex[user] = i
        i += 1
    if item not in itemIndex:
        itemIndex[item] = j
    lines.append(str(userIndex[user])+','+ str(itemIndex[item]))

